*```Author```*: ```Adedoyin Simeon Adeyemi```

# Hourly Rate Prediction of Employees

Regression Algorithms Used: RFR (Random Forest Regressor), SVR (Support Vector), LR (Linear Regression) and RR (Ridge Regressor)


# Employee Clustering

Clustering Algorithms Used: KMeans, DBSCAN


# Classifying Employee as either "well-paid" or "not-well-paid"

Classification Algorithms Used: RFC (Random Forest Classifier), SVC (Support Vector Classifier), GBC (Gradient Bossting Classifier)

# Import Libraries

In [ ]:
# import libraries

# Utilities
import os
import pickle
from math import sqrt

# Data wrangling and loader
import pandas as pd
import numpy as np

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py

# Data preprocessing
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from pandas import get_dummies

# Regressors
# import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge # Ridge Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR

# Clustering algorithm
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA # for dimension reduction
from sklearn.neighbors import NearestNeighbors

# Classifiers
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Performance metrics
## regression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
## classification
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score
# clustering
from sklearn.metrics import silhouette_score

# Suppressing unnecessary warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sns.set_style('darkgrid')
plt.rcParams['figure.figsize']=(15,8)
plt.rcParams['font.size']=18

# Connecting to Google Drive

In [ ]:
# Connecting Google Colab to Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
PWD = "gdrive/MyDrive/Colab Notebooks/Hourly_Rate_Prediction_Project"
DATASET_PATH = f'datasets'
MODEL_PATH = f'models'
RESULTS_PATH = f'results'

# Setting current working directory to project folder


In [ ]:
pwd

In [ ]:
%cd {PWD}

In [ ]:
# Creating appropriate folders if not already existing
if not os.path.exists(DATASET_PATH):
    os.mkdir(DATASET_PATH)

if not os.path.exists(MODEL_PATH):
    os.mkdir(MODEL_PATH)

if not os.path.exists(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

# Utility functions to save and load trained models

In [ ]:
def save_model_pickle(model, filename):
    try:
        pickle.dump(model, open(f'{MODEL_PATH}/{filename}', 'wb'))
        print('Saved')
    except Exception as err:
        print(err)


# Loading saved Pickle model
def load_model_pickle(filename):
    try:
        model = pickle.load(open(f'{MODEL_PATH}/{filename}', 'rb'))
        return model
    except Exception as err:
        print(err)
        return None

# Loading Dataset

# Data Source
- [Gender Pay Gap Dataset - Kaggle](https://www.kaggle.com/datasets/fedesoriano/gender-pay-gap-dataset)

- Feature Description


In [ ]:
# reading dataset
df1 = pd.read_csv(f'{DATASET_PATH}/CurrentPopulationSurvey.csv')
df1.head()

In [ ]:
# Generating employee_id column
df1['employee_id'] = list(range(1, len(df1)+1))

# EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
df1.shape

In [ ]:
#check shape
print(f"Total No of Rows: {df1.shape[0]} and Columns: {df1.shape[1]}")

In [ ]:
# basic info
df1.info()

In [ ]:
# total no of unique values in each columns
df1.nunique()

In [ ]:
# checking for null values
df1.isnull().sum()

In [ ]:
# Columns containing at least one null value(s)
df1.isnull().sum()[df1.isnull().sum() > 0.0]

In [ ]:
# null values in %
(df1.isnull().mean()[df1.isnull().mean() > 0.0] * 100).round(2)

In [ ]:
# Plotting Null Ratio for nullable columns
missing_ratio = (df1.isnull().mean()[df1.isnull().mean() > 0.0] * 100).round(2).sort_values(ascending=False)
pd.DataFrame(missing_ratio, columns=['Missing ratio %']).plot.bar()

In [ ]:
# plt.figure(figsize=(12,7))
# def plot_nas(df1: pd.DataFrame):
#     if df1.isnull().sum().sum() != 0:
#         na_df1 = ((df1.isnull().sum() / len(df1)) * 100).round(2)      
#         na_df1 = na_df1.drop(na_df1[na_df1 == 0].index).sort_values(ascending=False)
#         missing_data = pd.DataFrame({'Missing Ratio %' :na_df1})
#         missing_data.plot(kind = "bar")
#         plt.show()
#     else:
#         print('No NAs found')
# plot_nas(df1)

In [ ]:
df1.isna().sum()[df1.isna().sum()>0].sort_values(ascending=False).plot(kind='bar')

# FEATURE ENGINEERING AND SELECTION

## Dropping some unnecessary columns

In [ ]:
df1.drop([
      'o_numprec','o_hwtsupp','o_gq','o_metro','o_metarea','o_county','o_farm',
      'o_month','o_pernum','o_wtsupp','o_relate','o_bpl','o_mbpl','o_fbpl','o_nativity',
      'o_educ99','o_schlcoll','o_occ1990','o_ind1990','o_occ1950','o_ind1950','o_classwkr',
      'o_occly','o_occ50ly','o_indly','o_ind50ly','o_classwly','o_wkswork1','o_wkswork2',
      'o_hrswork','o_incwage','o_incbus','o_incfarm','o_inclongj','o_oincwage','o_srcearn',
      'o_ftype','o_quhrswor','o_qwkswork','o_qincbus','o_qincfarm','o_qincwage'
    ],axis = 1, inplace = True)

## Checking for duplicate records

In [ ]:
# checking duplicated values
np.all(df1.duplicated()) or 'No duplicate rows'

In [ ]:
# chexking duplicated values
df1.duplicated().sum()

In [ ]:
df1.info()

In [ ]:
df1.drop('o_statefip',axis = 1, inplace = True)

In [ ]:
df1.drop('o_yrimmig',axis = 1, inplace = True)

In [ ]:
df1_records_dropped = df1.dropna(axis=0, how='any')
df1_records_dropped.info()

In [ ]:
# statasic analysis
df1.describe().T

## More Exporatory Data Analysis (EDA)

In [ ]:
for col in df1.columns:
    print(col, df1[col].unique(), sep=":\n", end = "\n\n")

In [ ]:
df_numeric = df1[['hrwage', 'realhrwage', 'o_uhrswork', 'o_age',
 'annhrs']]
for i in df_numeric.columns:
 plt.figure(figsize=(5,2))
 sns.histplot(df1[i], bins = 10, kde = True, palette='hls')
 plt.xticks(rotation = 90)
 plt.show()


In [ ]:
# for i in df_categorical.columns:
#  plt.figure(figsize=(7,3))
#  sns.boxplot(df1[i], palette='hls')
#  plt.xticks(rotation = 90)
#  plt.show()

In [ ]:
df1.drop(['o_labforce','niincwage','incwageman','tcoincwage','tcinclongj','tcincwage'],axis = 1, inplace = True)

In [ ]:
# statasic analysis
df1.describe().T

In [ ]:
df1.drop('o_ind',axis = 1, inplace = True)

In [ ]:
# They are single-valued variables (columns), won't contribute anything
df1.drop(['selfemp','military','employed','groupquar','origrace','potexp2','expendbase10'],axis = 1, inplace = True)

In [ ]:
# statasic analysis
df1.describe().T

In [ ]:
# plot melted dataframe in a single command
sns.histplot(df_numeric.melt(), x='value', hue='variable',
             multiple='dodge', shrink=.75, bins=10);

## Correlation analysis for feature selection purposes

In [ ]:
corr_matrix = df1.corr().abs()
#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
                  .stack()
                  .sort_values(ascending=False))

#first element of sol series is the pair with the biggest correlation

In [ ]:
sol

In [ ]:
plt.figure(figsize=(14,10))
dfCorr = df1.corr()
filteredDf = dfCorr[((dfCorr >= .5) | (dfCorr <= -.5)) & (dfCorr !=1.000)]
plt.figure(figsize=(30,10))
sns.heatmap(filteredDf, annot=True, cmap="Reds")
plt.show()

#In the end, I created a small function to create the correlation matrix, filter it, and then flatten it. 

# As an idea, could easily be extended
e.g. asymmetric upper and lower bounds, etc.

In [ ]:
def corrFilter(df1: pd.DataFrame, bound: float):
    df1Corr = df1.corr()
    df1Filtered = df1Corr[((df1Corr >= bound) | (df1Corr <= -bound)) & (df1Corr != 1.000)]
    df1Flattened = df1Filtered.unstack().sort_values().drop_duplicates()
    return df1Flattened

res = corrFilter(df1, .7)
res

## Accesing Multi-Collinearity among features

**In general, an absolute correlation coefficient of >0.7 among two or more predictors indicates the presence of multicollinearity. Therefore, one of the following features will be dropped for the other.**

In [ ]:
first = [x for x,_ in res.index]
second = [y for _,y in res.index]
multi_col_attr = pd.DataFrame(zip(first, second), columns=['first', 'second'])

In [ ]:
multi_col_attr.head(20)

In [ ]:
multi_col_attr.iloc[20:40]

In [ ]:
multi_col_attr.iloc[40:60]

**Some important columns I always want to retain**

In [ ]:
COLS_TO_EXEMPT = ['educ99','sch', 'uhrswork', 'white', 'black', 'hisp', 'othrace', 'ba', 'adv', 'realhrwage', 
                  'northeast','northcentral', 'south','west', 'classwkr', 'citizen', 'ft', 'employee_id', 'inclongj'
                  ]

# Selected Columns that must be included
print(COLS_TO_EXEMPT)

### More Feature Engineering and Selection (Contn'd)

In [ ]:
# Selecting columns to be dropped
cols_to_drop = [
    'perconexp','year','wagesamp','race', 'educorig', 'region', 'ind_orig', 'farmer', 'ind','serial', 'qwkswork',
    'fbpl','o_region', 'bpl', 'occ1990','occ_1990', 'occ_1999', 'mbpl', 'ftype', 'relate', 'ind_2007_orig', 'ind2000_90', 'ind2000_99',
    'o_hispan', 'ind50ly', 'ind_2002_orig', 'ind_2000', 'classwly', 'occ1990', 'adj_ind', 'ind_1990', 'occ2000_90', 'occ2000_99', 
    'ind_2007_orig', 'ind_1999', 'o_citizen', 'ind1950', 'ind_2002_orig', 'o_race', 'ind_81', 'o_uhrswork', 'lnrwg'
]

for i in range(len(multi_col_attr)):
    curr = multi_col_attr.iloc[i]
    first = curr['first']
    sec = curr['second']

    if first not in cols_to_drop and first not in COLS_TO_EXEMPT:
        cols_to_drop.append(first)
    elif sec not in cols_to_drop and sec not in COLS_TO_EXEMPT:
        cols_to_drop.append(sec)


In [ ]:
# Multi-collinear columns to be deleted
sorted(cols_to_drop.copy())

In [ ]:
# Dropping one of the multi-collinear columns
print(f'Number of Collinnear columns to drop: {len(cols_to_drop)}')
df1.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.info()

## Dropping columns having more than 30% missing data and columns having a single value data points

In [ ]:
cols_with_over_30_perc_missing_data = df1.isna().mean()[df1.isna().mean() * 100 > 30.0] * 100
cols_with_over_30_perc_missing_data

In [ ]:
more_cols_to_drop = list(cols_with_over_30_perc_missing_data.index)

# Drop single valued columns
for col in df1.columns:
    if df1[col].nunique() < 2 and col not in more_cols_to_drop:
        more_cols_to_drop.append(col)

# exempt the selected column names
for col in COLS_TO_EXEMPT:
    if col in more_cols_to_drop:
        more_cols_to_drop.remove(col)

print(f'No of more columns to drop: {len(more_cols_to_drop)}')
print(more_cols_to_drop)

In [ ]:
df1.drop(columns=more_cols_to_drop, inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.info()

## Dropping more columns based on understanding of data captured as described on the dataset kaggle webpage

In [ ]:
df_corr = df1.drop(columns='adj_occ2name').corr()
ab = df1.drop(columns='adj_occ2name').corr()[((df_corr > 0.5) & (df_corr < 1.0))].dropna(thresh=1, axis=1).replace(np.NaN, '-')
ab.dropna(thresh=1, axis=0)

**Selected Columns that must always be included, selected based on domain knowledge**

In [ ]:
# Selected Columns that must always be included, selected based on domain knowledge, as described on data homepage
print(COLS_TO_EXEMPT)

In [ ]:
df1.info()

### More columns to remove based on domain knowledge of dataset (Kaggle dataset attribute descriptions)

In [ ]:
# more_cols_to_drop = ['hwtsupp', 'statefip', 'qincfarm', 'o_educ']

more_cols_to_drop = ['adj_occ2name', 'o_marst', 'o_empstat', 'o_union', 'union', 'oincwage', 'hrswork',
      'srcearn', 'hdwfcoh', 'un_lnrealwg', 'numprec', 'pernum', 'qincfarm', 'wtsupp', 'o_age', 'o_occ',
      'hrwage'
    ]

In [ ]:
# more_cols_to_drop = ['hrwage']

In [ ]:
df1.info()

In [ ]:
df1.drop(columns=more_cols_to_drop, inplace=True)

In [ ]:
df1.info()

# DATA PREPROCESSING

**Having selected relevant features, the result of Feature Engineering carried out, We now have a working dataset to be preprocessed and used**

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
print(f'Total Observations (Rows): {df1.shape[0]}')
print(f'Total Variable size (Columns): {df1.shape[1]}')

# Exploring The Data to know how to preprocess it

In [ ]:
df1.head()

## Exploring Dummy and Non-Dummy Variables (Columns)

In [ ]:
# Viewing all non-dummy variables
non_dummy_cols = []
for col in df1.columns[:-1]:
    unique_values = df1[col].unique()
    if not (len(unique_values) == 2 and 1 in unique_values and 0 in unique_values):
        non_dummy_cols.append(col)

print('Non-Dummy Columns: ')
print(non_dummy_cols)

In [ ]:
# Dummy Variables (Columns)
df1.drop(columns=non_dummy_cols).info()

---
**NOTE:** 
- For dummy variables it can be observed that there are no missing data and obviously no cleaning necessary, since all columns contains values [0,1]

---

In [ ]:
# Non-Dummy Variables (columns)
df1[non_dummy_cols].head()

In [ ]:
df1[non_dummy_cols].info()

In [ ]:
df_non_dummy = df1[non_dummy_cols]

### Treating missing data (nullables) 

In [ ]:
df_non_dummy.isna().sum()

#### Metro

In [ ]:
# % of missing data
print(f'% of missing data in "metro" column: {np.round(df_non_dummy.metro.isna().mean() * 100, 2)}%')

In [ ]:
df_non_dummy.metro.value_counts()

In [ ]:
df_non_dummy.metro.fillna('others', inplace=True)
df_non_dummy.metro.value_counts()

#### Citizen

In [ ]:
# % of missing data
print(f'% of missing data in "citizen" column: {np.round(df_non_dummy.citizen.isna().mean() * 100, 2)}%')

---
**NOTE:**
- We decided not to drop "citizen" column despite its large number of missing values because we consider it an important feature that may affect wage(s)
---

In [ ]:
df_non_dummy.citizen.value_counts()

In [ ]:
# Filling missing values with 'others'
df_non_dummy.citizen.fillna('others', inplace=True)
df_non_dummy.citizen.value_counts()

#### educ99

In [ ]:
# % of missing data
print(f'% of missing data in "educ99" column: {np.round(df_non_dummy.educ99.isna().mean() * 100, 2)}%')

In [ ]:
df_non_dummy.educ99.value_counts()

In [ ]:
# Filling missing values with 'others'
df_non_dummy.educ99.fillna('others', inplace=True)
df_non_dummy.educ99.value_counts()

#### inclongj

In [ ]:
# % of missing data
print(f'% of missing data in "inclongj" column: {np.round(df_non_dummy.inclongj.isna().mean() * 100, 2)}%')

In [ ]:
df_non_dummy.inclongj.value_counts()

In [ ]:
# Filling missing values with mean of income of longest job taken
avg_income = df_non_dummy.inclongj.mean()
df_non_dummy.inclongj.fillna(avg_income, inplace=True)

#### Confirming that all missing data have been treated

In [ ]:
# Confirming that all missing data have been treated
df_non_dummy.isna().sum()

# Encoding categorical columns using One-Hot (dummy variables)

In [ ]:
for col in df_non_dummy:
    print(col, df_non_dummy[col].unique(), sep=':\n', end='\n\n')

In [ ]:
# Encoding Categorical columns
cat_cols = ['metro', 'sex', 'marst', 'citizen', 'empstat', 'classwkr']

# Changing datatypes of columns into categorical-type first
for col in cat_cols:
    df_non_dummy[col] = df_non_dummy[col].astype(np.str)

encoded_cats = get_dummies(df_non_dummy[cat_cols], drop_first=True)
encoded_cats.head()

In [ ]:
encoded_cats.shape

In [ ]:
# Replacing the original columns with the encoded columns
df_non_dummy_encoded = pd.concat([df_non_dummy.drop(columns=cat_cols), encoded_cats], axis='columns')
df_non_dummy_encoded.head()

In [ ]:
# Replacing 'others' values for educ99 with 0.0
df_non_dummy_encoded.educ99 = df_non_dummy_encoded.educ99.replace('others',0.0)
df_non_dummy_encoded.head()

In [ ]:
df_non_dummy_encoded.info()

## Merging encoded Non-Dummy dataframe with the dummy variables

In [ ]:
## Merging encoded Non-Dummy dataframe with the dummy variables
cleaned_dataset = pd.concat([df1.employee_id, df_non_dummy_encoded, df1.drop(columns=non_dummy_cols).iloc[:,:-1]], axis='columns')
cleaned_dataset.head()

In [ ]:
cleaned_dataset.info()

# Analyzing Non-Dummy Variables

In [ ]:
cleaned_dataset.head()

In [ ]:
data_to_analyze = cleaned_dataset.iloc[:,:8]
data_to_analyze

In [ ]:
data_to_analyze.iloc[:, 1:].describe()

In [ ]:
data_to_analyze.age.plot.hist(title='Age Distribution\n')
plt.xlabel('Age')

In [ ]:
data_to_analyze.sch.value_counts().plot.bar(title='sch Data Distribution\n')
plt.xlabel('Education Attainment Codes')
plt.ylabel('Value Counts')

**sch:** educLbl Educational attainment recode: 

(None=0, 1=1, Grades 1=2, 2.5=2.5, 3=3, 4=4, Grades 5=5, 5.5=5.5, 6=6, Grades 7=7, 7.5=7.5, 8=8, Grade 9=9, Grade 10=10, Grade 11=11, Grade 12=12, Some Coll=13, Assoc.=14, BA=16, Adv. Degr=18)

In [ ]:
data_to_analyze.educ99.value_counts().plot.bar(title='educ99 Data Distribution\n')
plt.xlabel('Education Attainment')
plt.ylabel('Value Counts')

**educ99:** 

educ99_lbl Educational attainment, 1990, available for 1999 and later: 

(No school=1, 1st-4th g=4, 5th-8th g=5, 9th grade=6, 10th grad=7, 11th grad=8, 12th grad=9, High scho=10, Some coll=11, Associate=13, Associate=14, Bachelors=15, Masters d=16, Professio=17, Doctorate=18, Novalue=0)

In [ ]:
data_to_analyze.wkswork1.plot.hist(title='Weeks worked last year (wkswork1) Data Distribution\n')
plt.xlabel('wkswork1')

In [ ]:
data_to_analyze.uhrswork.plot.hist(title='Usual hours worked per week (last yr) (uhrswork) Data Distribution\n')
plt.xlabel('uhrswork')

In [ ]:
data_to_analyze.inclongj.plot.hist(title='Earnings from longest job (inclongj) Data Distribution\n')
plt.xlabel('inclongj')

In [ ]:
# Target Variable (realhrswage) Data Distribution
data_to_analyze.realhrwage.plot.hist(title='Real Hourly Wage, inflated to 2010 dollars (realhrswage) - Data Distribution\n')
plt.xlabel('realhrswage')

# Exploring Outliers

In [ ]:
for col in data_to_analyze.iloc[:,1:].columns:
    # data_to_analyze[col].plot(kind='box', title=f'{col} Outlier Analysis')
    # plt.title(f'{col} Outlier Analysis')
    plt.figure(figsize=(5,2))
    sns.boxplot(data_to_analyze[col], palette='hls')
    plt.title(f'{col} Outlier Analysis\n')
    plt.xticks(rotation = 90)
    plt.show()
    print('\n\n')

# Exploring correlation of variables with Target variable (realhrwage)

In [ ]:
data_to_analyze.iloc[:, 1:-1].corrwith(data_to_analyze.realhrwage).sort_values(ascending=False)

In [ ]:
# Visualizing Feature Correlation with Target Variable
plt.figure(figsize=(10,4))
data_to_analyze.iloc[:, 1:-1].corrwith(
    data_to_analyze.realhrwage
    ).sort_values(
        ascending=False
        ).plot.bar(
            title='Features Correlation with Target Variable (realhrwage)\n')

In [ ]:
# Setting the index to 'employee_id'
cleaned_dataset.index = cleaned_dataset.employee_id
cleaned_dataset.drop(columns=['employee_id'], inplace=True)

In [ ]:
# Saving a copy of the cleaned dataset
cleaned_dataset.to_csv(f'{RESULTS_PATH}/cleaned_dataset.csv')

# Data Segmentation and Scaling

In [ ]:
# Loading the cleaned dataset
cleaned_dataset = pd.read_csv(f'{RESULTS_PATH}/cleaned_dataset.csv', index_col=0)
cleaned_dataset.head()

In [ ]:
cleaned_dataset.info()

In [ ]:
X = cleaned_dataset.drop(columns=['realhrwage'])
y = cleaned_dataset.realhrwage

In [ ]:
X.head(3)

In [ ]:
y.head(3)

## Scaling the features (X)

Scaling is essential in this case because of some large-valued features. These features may dominate the others with smaller values, especially the dummy variables, rendering them insignificant.

In [ ]:
scaler = MinMaxScaler()
scaledX = scaler.fit_transform(X)
scaledX = pd.DataFrame(scaledX, columns=X.columns, index=X.index)
scaledX.head()

## Segmenting into training and testing sets

**Split ratio: (70:30)**

training set: 70%,
testing set: 30%

In [ ]:
# Segmenting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaledX, y, test_size=0.30, random_state=42)

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
X_test.head(3)

In [ ]:
y_test.head(3)

In [ ]:
print(f'Shape of X-train: {X_train.shape}')
print(f'Shape of X-test: {X_test.shape}')

# REGRESSION

## Random Forest Regressor

In [ ]:
# instantiation
rfr = RandomForestRegressor(
    n_estimators=100, criterion='squared_error', 
    bootstrap=True, random_state=60)

# Training
rfr.fit(X_train, y_train)

# predictions
rfr_pred = rfr.predict(X_test)


In [ ]:
# Evaluations
rfr_rmse = sqrt(mean_squared_error(y_test, rfr_pred))
rfr_mae = mean_absolute_error(y_test, rfr_pred)

print(f'RFR RMSE: {rfr_rmse}')
print(f'RFR MAE: {rfr_mae}')

## Support Vector Machine (Regressor)

In [ ]:
# instantiation
svr = LinearSVR(epsilon=0.0, tol=0.0001, C=1.0, fit_intercept=True, random_state=60)

# Training
svr.fit(X_train, y_train)


In [ ]:
# predictions
svr_pred = svr.predict(X_test)

# Evaluations
svr_rmse = sqrt(mean_squared_error(y_test, svr_pred))
svr_mae = mean_absolute_error(y_test, svr_pred)

print(f'SVR RMSE: {svr_rmse}')
print(f'SVR MAE: {svr_mae}')

## Linear Regression

In [ ]:
# instantiation
lr = LinearRegression(fit_intercept=True)

# Training
lr.fit(X_train, y_train)

In [ ]:
# predictions
lr_pred = lr.predict(X_test)

# Evaluations
lr_rmse = sqrt(mean_squared_error(y_test, lr_pred))
lr_mae = mean_absolute_error(y_test, lr_pred)

print(f'LR RMSE: {lr_rmse}')
print(f'LR MAE: {lr_mae}')

## Ridge Regressor

**Ridge Regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of the coefficients with l2 regularization.**

It is a Linear least squares with l2 regularization.

It Minimizes the objective function::

||y - Xw||^2_2 + alpha * ||w||^2_2

This model solves a regression model where the loss function is
the linear least squares function and regularization is given by
the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
This estimator has built-in support for multi-variate regression
(i.e., when y is a 2d-array of shape (n_samples, n_targets)).



In [ ]:
# instantiation
rr = Ridge(alpha=1.0, tol=0.0001, fit_intercept=True, random_state=60)

# Training
rr.fit(X_train, y_train)

In [ ]:
# predictions
rr_pred = rr.predict(X_test)

# Evaluations
rr_rmse = sqrt(mean_squared_error(y_test, rr_pred))
rr_mae = mean_absolute_error(y_test, rr_pred)

print(f'RR RMSE: {rr_rmse}')
print(f'RR MAE: {rr_mae}')

### Regression Comparative Performance Evaluations

In [ ]:
REGRESSION_METRICS = ['RMSE', 'MAE']
regression_results = {
    'RFR': [rfr_rmse, rfr_mae],
    'SVR': [svr_rmse, svr_mae],
    'LR': [lr_rmse, lr_mae],
    'RR': [rr_rmse, rr_mae],

}

regression_results = pd.DataFrame(regression_results, index=REGRESSION_METRICS)
regression_results

In [ ]:
regression_results.T.sort_values(by='RMSE').T.plot.bar(title='Regression Comparative Performance Evaluation (Errors)\n')
plt.xlabel('Error Type')
plt.ylabel('Error Value')


**Interpretation of Regression Results**

As shown above, Support Vector (SVR) model performed better than Ridge Regression (RR), Linear Regression (LR) and Random Forest Regressor (RFR) in terms of Root Mean squared Error (RMSE) with about 21.44, 3.21 and 3.24 reduction in error measured in terms of RMSE over RR, RR and LR respectively. 

The implication of this is that SVR model's prediction of *Hourly Wage* is accurate to a +/- $41.97 error rate.


It can also be observed that Ridge Regression (RR) Model performed slightly better than Linear Regression (LR) with 0.3 performance improvement in terms of both RMSE and MAE (Mean Absolute Error).


In Terms of Mean Absolute Error, Random Forest Regressor (RFR) model performed better than the rest of the other model, with 1.7, 9.05, and 9.02 reduction in error compared to SVR, LR and RR models respectively.


Considering the marginal difference of 1.7 MAE reduction (which is an improvement) of RFR over SVR; and that of 21.44 RMSE reduction (which is an improvement) of SVR over RFR, it can be concluded that SVR model performed better than all other models in predicting hourly wage of employee.

### Regression: Accessing Feature Importances for Regression Case

In [ ]:
feature_importances = pd.DataFrame(rfr.feature_importances_, columns=['importance'], index=X_train.columns).sort_values(by='importance', ascending=False)
feature_importances

In [ ]:
# plotting top 20 important features
feature_importances[:20].plot.bar(title='Feature Importances\n')
plt.xlabel('Features')
plt.ylabel('Importances')

## Saving regression results and trained models

In [ ]:
# Saving a copy of regression results
regression_results.to_csv(f'{RESULTS_PATH}/regression_results.csv')

# Saving trained models
save_model_pickle(rfr, filename='RFR_MODEL.pickle')
save_model_pickle(svr, filename='LinearSVR_MODEL.pickle')
save_model_pickle(lr, filename='LR_MODEL.pickle')
save_model_pickle(rr, filename='RR_MODEL.pickle')

# saving 'scaler' object for regression
save_model_pickle(scaler, filename='SCALAR_Regression.pickle')


---
# CLUSTERING
---

---
## KMeans

## Loading saved cleaned dataset

In [ ]:
cleaned_dataset = pd.read_csv(f'{RESULTS_PATH}/cleaned_dataset.csv', index_col=0)
cleaned_dataset.head()

## Scaling for clustering by including the 'realhrwage' variable

In [ ]:
# making a copy of the cleaned_dataset in case we make changes to it
XC = cleaned_dataset.copy()
XC.head()

In [ ]:
# Scaling
scaler_clust = MinMaxScaler()
scaledXC = scaler_clust.fit_transform(XC.iloc[:,:7])
scaledXC = pd.DataFrame(scaledXC, columns=XC.columns[:7], index=XC.index)
scaledXC.head()

# KMeans Implementation

## Finding the best cluster size (k-value) using Elbow Method

with the within-cluster-sum-of-squared (wcss) values

In [ ]:
# Using the elbow method to find the optimal number of clusters (k-value)

WCSS = []
for i in range (1,  20):
    kmeans = KMeans(n_clusters=i, init="k-means++", random_state=50)
    kmeans.fit(scaledXC)
    WCSS.append(kmeans.inertia_)
    
plt.plot(range(1, 20), WCSS)
plt.title("The Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")

**From the graph, k=4 seems a good k value (elbow point) at which point further division brings less significant reduction in intra-cluster distance (wcss) measure**

### Training KMeans on data

### Using K-Means at k = 4 (no of clusters)

In [ ]:
K = 4

In [ ]:
kmeans = KMeans(n_clusters=K, init="k-means++", random_state=50)
y_kmeans = kmeans.fit_predict(scaledXC)

In [ ]:
y_kmeans[:10]

In [ ]:
scaledXC.shape

## Dimensionality Reduction

**Using Principal Component Analysis (PCA)**


7 columns is way too much dimension to plot, therefore using PCA for dimensionality reduction is essential to be able to visualize the clusters on 2-D plane

In [ ]:
pca = PCA(n_components=2)
reduced_scaledXC = pca.fit_transform(scaledXC)
reduced_scaledXC = pd.DataFrame(reduced_scaledXC, columns=['x1', 'x2'], index=scaledXC.index)
reduced_scaledXC.head()

In [ ]:
# Adding the clusters columns
reduced_scaledXC['kmeans_cluster'] = y_kmeans
reduced_scaledXC.head()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print(f'Explained Variance: {sum(pca.explained_variance_ratio_)}')

**About 78% of the original variance is explained by the 2 new dimensions retained.**

This is a good-enough representation of the data, considering the high dimension (7) involved, and for visualization purposes, we can use it.

### Visualizing the clusters

In [ ]:
COLORS = [
    'red','green','blue','yellow','brown','pink','silver','black',
    'tomato', 'grey','indigo','violet','blue','purple','green','blue',
    'cyan','brown','orange','pink','gold'
]

plt.figure(figsize=(10,8))
for i in range(K):
    plt.scatter(reduced_scaledXC[reduced_scaledXC.kmeans_cluster == i].x1.values, 
                reduced_scaledXC[reduced_scaledXC.kmeans_cluster == i].x2.values, 
                s=20, c=COLORS[i], label=f'Cluster_{i}')

plt.title('Cluster of Employees using K-Means (scaled dataset)')
plt.xlabel('Dimesion 1')
plt.ylabel('Dimesion 2')
plt.legend(loc='lower right')

In [ ]:
# evaluations

# wcss
km_wcss = kmeans.inertia_
print(f'WCSS for KMeans: {km_wcss}')

In [ ]:
reduced_scaledXC.kmeans_cluster.value_counts()

**Silhouette Score**

This score is the mean Silhouette Coefficient of all samples.


The Silhouette Coefficient is calculated using the mean intra-cluster
distance (``a``) and the mean nearest-cluster distance (``b``) for each sample.  


The Silhouette Coefficient for a sample is ``(b - a) / max(a,
b)``.  


- To clarify, ``b`` is the distance between a sample and the nearest cluster that the sample is not a part of.

**Note** that Silhouette Coefficient is only defined if number of labels is 2 <= n_labels <= n_samples - 1.


This returns the mean Silhouette Coefficient over all samples.
To obtain the values for each sample, use :func:`silhouette_samples`.

The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. 

Negative values generally indicate that a sample has
been assigned to the wrong cluster, as a different cluster is more similar.

In [ ]:
# 4 clusters
km_silhouette_score = silhouette_score(scaledXC, y_kmeans)
print(f'Silhouette Score of KMeans: {km_silhouette_score}')

In [ ]:
clustering_data = XC.iloc[:,:7]
clustering_data['kmeans_clusters'] = y_kmeans

In [ ]:
clustering_data.head()

## Analyzing KMeans Clustered Groups

In [ ]:
clustering_data.kmeans_clusters.value_counts()

In [ ]:
cluster0_km = clustering_data[clustering_data.kmeans_clusters == 0]
cluster1_km = clustering_data[clustering_data.kmeans_clusters == 1]
cluster2_km = clustering_data[clustering_data.kmeans_clusters == 2]
cluster3_km = clustering_data[clustering_data.kmeans_clusters == 3]

### Cluster 0 - Kmeans

In [ ]:
cluster0_km.head()

In [ ]:
print('Cluster 0 Employees Group Analysis')
cluster0_km.describe()

### Cluster 1 - Kmeans

In [ ]:
cluster1_km.head()

In [ ]:
print('Cluster 1 Employees Group Analysis\n\n')
cluster1_km.describe()

### Cluster 2 - Kmeans

In [ ]:
cluster2_km.head()

In [ ]:
print('Cluster 2 Employees Group Analysis')
cluster2_km.describe()

### Cluster 3 - Kmeans

In [ ]:
cluster3_km.head()

In [ ]:
print('Cluster 3 Employees Group Analysis')
cluster3_km.describe()

**Interpretation of Cluster Results**

The result shows a silhouette score of 0.34 which indicates how compact points within clusters are and how far away point across clusters are. It measures the ratio of intra-cluster to inter cluster distances, ranging between -1 and 1, with -1 being the worst clustering and 1 being the best. 

The result of 0.34 shows a good cluster. Although it is slightly close to 0, showing a very small amount of overlaping clusters.

---
# DBSCAN
**Clustering Algorithm2**


### Finding the best epsilon value for the dataset using nearest neighbors

In [ ]:
neighbors = NearestNeighbors(n_neighbors=2)
neighbors.fit(scaledXC)

In [ ]:
distances, indices = neighbors.kneighbors(scaledXC)

In [ ]:
distances

In [ ]:
indices

In [ ]:
distances = distances[:,1]
distances.sort(axis=0)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(distances)

plt.title('Finding The Best Epsilon Value')

Since the distances started increasing at about ~0.06, we pick that as the epsilon value
epsilon = 0.06

In [ ]:
epsilon = 0.06
dbscan = DBSCAN(eps=epsilon, min_samples=5)
y_dbscan = dbscan.fit_predict(scaledXC)

In [ ]:
y_dbscan[:10]

In [ ]:
# Adding the DBSCAN clusters columns
reduced_scaledXC['dbscan_cluster'] = y_dbscan
reduced_scaledXC.head()

In [ ]:
print(f'No of Clusters (DBSCAN): {reduced_scaledXC.dbscan_cluster.nunique()}')

### Visualizing DBSCAN Clusters

In [ ]:
n_clusters = y_dbscan.max() + 1

plt.figure(figsize=(10,8))

for i in range(n_clusters):
    plt.scatter(reduced_scaledXC[reduced_scaledXC.dbscan_cluster == i].x1.values, 
                reduced_scaledXC[reduced_scaledXC.dbscan_cluster == i].x2.values,  
                s=20, #c=COLORS[i], 
                label=f'Cluster_{i}')

plt.title('Cluster of Employees using DBSCAN (Scaled Dataset)\n')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend(loc='lower right')

In [ ]:
# Adding the DBSCAN clusters columns to clusters dataset 
clustering_data['dbscan_cluster'] = y_dbscan

clustering_data.head()


## Analyzing DBSCAN Clustered Groups

In [ ]:
clustering_data.dbscan_cluster.value_counts()

In [ ]:
cluster0_db = clustering_data[clustering_data.dbscan_cluster == 0]
cluster1_db = clustering_data[clustering_data.dbscan_cluster == 1]
cluster2_db = clustering_data[clustering_data.dbscan_cluster == 2]
cluster3_db = clustering_data[clustering_data.dbscan_cluster == 3]

### Cluster 0 - DBSCAN

In [ ]:
cluster0_db.head()

In [ ]:
print('Cluster 0 Employees Group Analysis')
cluster0_db.describe()

### Cluster 1 - DBSCAN

In [ ]:
cluster1_db.head()

In [ ]:
print('Cluster 1 Employees Group Analysis\n\n')
cluster1_db.describe()

### Cluster 2 - DBSCAN

In [ ]:
cluster2_db.head()

In [ ]:
print('Cluster 2 Employees Group Analysis')
cluster2_db.describe()

### Cluster 3 - Kmeans

In [ ]:
cluster3_db.head()

In [ ]:
print('Cluster 3 Employees Group Analysis')
cluster3_db.describe()

In [ ]:
silhouette_score_db = silhouette_score(scaledXC, y_dbscan)

In [ ]:
print(f'DBSCAN Silhouette Score: {silhouette_score_db}')

DBSCAN Silhouette Score: -0.2685777444300517

**Interpretation of Cluster Results**

The result shows a silhouette score of -0.27 which indicates how compact points within clusters are and how far apart points across clusters are. It measures the ratio of intra-cluster to inter cluster distances, ranging between -1 and 1, with -1 being the worst clustering and 1 being the best. 

The result of -0.27 shows a very poor clustering. Although it is slightly close to 0, The negative value shows higher average intra-cluster distances than average inter-cluster distances, showing a considerable  amount of overlaping clusters.

It can be observed that KMeans clustering model did way better than DBSCAN with a silhouette score of 0.34 compared to -0.27 silhouette score of DBSCAN.

It can also be observed that DBSCAN produced too many clusters up to 286 clusters, which is too large and contains a lot of overlapping clusters as shown in the visualized clusters (for DBSCAN).

In [ ]:
# saving the KMeans trained model
save_model_pickle(kmeans, filename='KMEANS_MODEL.pickle')

# saving the DBSCAN trained model
save_model_pickle(dbscan, filename='DBSCAN_MODEL.pickle')

# saving clustering 'scaler' object
save_model_pickle(scaler_clust, filename='SCALER_for_CLUSTERING.pickle')

# Saving a copy of the clusters dataset along with the KMeans & DBSCAN clustered labels
clustering_data.to_csv(f'{RESULTS_PATH}/clustered_Dataset.csv')


---
# CLASSIFICATION
---

This task is to classifier employee into ```well-paid``` (Hourly wage above Federally approved US minimum wage of $7.50) and ```not-well-paid``` (hourly wage below minimum wage of $7.50).

These classes will be encoded as:
```bash
{0: not-well-paid, 1: well-paid}
```

## My custom class encoder_decoder utility function

In [ ]:
def encode_wage(hourly_wage):
  min_wage = 7.50
  if hourly_wage < min_wage:
    return 0
  else:
    return 1


def encode_class(class_name):
  if class_name == 'not-well-paid':
      return 0
  elif class_name == 'well-paid':
      return 1
  else:
      raise Exception('Class name passed not recognized, valid values are {"well-paid", "not-well-paid"}')


def decode_class(class_label):
    if class_label == 0:
        return 'not-well-paid'
    elif class_label == 1:
        return 'well-paid'
    else:
        raise Exception('Invaid class label. Valid labels are {0, 1}')

In [ ]:
CLASSIFICATION_METRICS = ['Accuracy', 'Precision', 'Recall']

## Data Segmentation for Classification

In [ ]:
XC.head()

In [ ]:
XClass = XC.drop(columns=['realhrwage'])
yClass = XC.realhrwage.apply(encode_wage)

In [ ]:
XClass.head()

In [ ]:
yClass.head()

In [ ]:
print(f'Shape of X: {XClass.shape}')

In [ ]:
XClass.info()

In [ ]:
yClass.value_counts()

In [ ]:
yClass.value_counts().plot.bar(title='Target Variable Distribution')

**It can be observed above that there is huge class imbalance that may introduce bias towards the majority class in our model.**

We therefore need to remove the imbalance. We chose to randomly sample the same size of observations for both classes.

In [ ]:
# Saving a copy of the dataset used for classification
yClass.name = 'hrwage_class_label'
classification_dataset = pd.concat([XClass, yClass], axis='columns')
classification_dataset.to_csv(f'{RESULTS_PATH}/classification_dataset.csv')

In [ ]:
classification_dataset.head()

## Moderating Class Imbalance (to avoid a possible bias towards the majority class)

**In order to remove a possible bias towards the well-paid (1) class due to class imbalance, we have chosen to select equal randomly sampled observations for both classes.**

In [ ]:
labels_0_data = classification_dataset[classification_dataset.hrwage_class_label == 0]
labels_1_data = classification_dataset[classification_dataset.hrwage_class_label == 1]

print(f'Class 0 Population size: {len(labels_0_data)}')
print(f'Class 1 Population size: {len(labels_1_data)}')

In [ ]:
sampled_labels_1_data = labels_1_data.sample(len(labels_0_data), random_state=60)

In [ ]:
bal_class_ds = pd.concat([labels_0_data, sampled_labels_1_data], axis='index')
print(f'Class 0 Population size: {len(labels_0_data)}')
print(f'Balanced Class 1 Population size: {len(sampled_labels_1_data)}')
print(f'Shape of balanced dataset: {bal_class_ds.shape}')


In [ ]:
# Reviewing Target Data Distribution
bal_class_ds.hrwage_class_label.value_counts().plot.bar(title='Balanced Target Variable Distribution (Moderated)')

In [ ]:
new_XClass = bal_class_ds.drop(columns=['hrwage_class_label'])
new_yClass = bal_class_ds.hrwage_class_label

In [ ]:
new_XClass.head()

In [ ]:
new_yClass.head()

## Scaling the features (X)

In [ ]:
scaler_classif = MinMaxScaler()
scaled_XClass = scaler_classif.fit_transform(new_XClass)
scaled_XClass = pd.DataFrame(scaled_XClass, columns=new_XClass.columns, index=new_XClass.index)
scaled_XClass.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_XClass, new_yClass, test_size=0.30, random_state=42)

In [ ]:
print(f'Shape of X: {new_XClass.shape}')
print(f'Shape of X-Train: {X_train.shape}')
print(f'Shape of X-Test: {X_test.shape}')

---
## Random Forest Classifier

In [ ]:
# instantiation
svc2 = SVC(C=1, kernel='rbf', tol=0.001, gamma='scale', probability=True, random_state=60)

# training
svc2.fit(X_train, y_train)

# predictions
svc2_pred = svc2.predict(X_test)

print(f'Classification Report for SVC2-Classifier: \n')
print(classification_report(y_test, svc2_pred))

In [ ]:
# instantiation
rf = RandomForestClassifier(n_estimators=100, random_state=60)

# training
rf.fit(X_train, y_train)

In [ ]:
# predictions
rf_pred = rf.predict(X_test)

# evaluations
rf_acc = accuracy_score(y_test, rf_pred)
rf_prec = precision_score(y_test, rf_pred, average='weighted')
rf_rec = recall_score(y_test, rf_pred, average='weighted')

rf_cf = confusion_matrix(y_test, rf_pred)

print(f'Classification Report for RF-Classifier: \n')
print(classification_report(y_test, rf_pred))

**Interpretation of the Classification Report for Random Forest Classifier**

As shown above, Random Forest classifier has an average precision, recall, f1-score and by implication accuracy of 0.91.

The lowest class prediction is 0.88 recall for ```well-paid (class 1)``` employees and 0.89 precision for ```non-well-paid (class 0)``` employees.

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(rf_cf, fmt='.4g', annot=True, cmap='Blues')
plt.title('\nConfusion Matrix for Random Forest Classifier\n')

In [ ]:
cf_perc = (rf_cf / rf_cf.sum(axis=0) * 100).round(1).reshape(2,2)

plt.figure(figsize=(6,4))
sns.heatmap(cf_perc, annot=True, fmt='.1f', cmap='Blues')
plt.title('\nConfusion Matrix for Random Forest Classifier (%)\n')

**Interpretation of the Confusion Matrix for Random Forest Classifier**

As shown above, the x-axis (vertical) indicate the true classes (clusters) and the y-axis (horizontal) indicates the predicted classes (clusters). The diagonal axis indicates number and corresponding percentages of correct classifications, while the rest are misclassifications.

Out of 7,911 employees belonging to class 0 (not-well-paid) presented for classification, 7,492 of them were correctly classified, constituting about 88.6% of the population, while 966 (11.4%) employees were mis-classified as belonging to class 1 (well-paid).

Also, of the 7,964 employees who are in class 1 (well-paid), 6,998 (94.4%)of them were correctly classified and the remaining 419 (5.6%) employees were miscalssified as class 0 (not-well-paid).

And so on. The overall performances shown that out of the 15,875 employees presented for classification, 14,490 (91.3%) of them were correctly classified leaving only about 9.7% misclassification.

In [ ]:
rf_results = {
    'RF': [rf_acc, rf_prec, rf_rec]
}
rf_results = pd.DataFrame(rf_results, index=CLASSIFICATION_METRICS)
rf_results

In [ ]:
plt.figure(figsize=(6,4))
rf_results.plot.bar(title='RF Results')

---
## Support Vector Classifier (SVC)

In [ ]:
# instantiation
svc = SVC(C=1, kernel='rbf', tol=0.001, gamma='scale', probability=True, random_state=60)

# training
svc.fit(X_train, y_train)

In [ ]:
# predictions
svc_pred = svc.predict(X_test)

# evaluations
svc_acc = accuracy_score(y_test, svc_pred)
svc_prec = precision_score(y_test, svc_pred, average='weighted')
svc_rec = recall_score(y_test, svc_pred, average='weighted')

svc_cf = confusion_matrix(y_test, svc_pred)

print(f'Classification Report for SVM-Classifier (SVC): \n')
print(classification_report(y_test, svc_pred))

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(svc_cf, fmt='.4g', annot=True, cmap='Blues')
plt.title('\nConfusion Matrix for Support Vector Classifier (SVC)\n')

In [ ]:
plt.figure(figsize=(6,4))
cf_perc = (svc_cf / svc_cf.sum(axis=0) * 100).round(1).reshape(2,2)
sns.heatmap(cf_perc, annot=True, fmt='.1f', cmap='Blues')
plt.title('\nConfusion Matrix for Support Vector Classifier (SVC) (%)\n')

**Interpretation of the Confusion Matrix for Support Vector Classifier (SVC)**

As shown above, the x-axis (vertical) indicate the true classes (clusters) and the y-axis (horizontal) indicates the predicted classes (clusters). The diagonal axis indicates number and corresponding percentages of correct classifications, while the rest are misclassifications.

Out of 7,911 employees belonging to class 0 (not-well-paid) presented for classification, 6,468 of them were correctly classified, constituting about 84.8% of the population, while 1,158 (15.2%) employees were mis-classified as belonging to class 1 (well-paid).

Also, of the 7,964 employees who are in class 1 (well-paid), 6,806 (82.5%)of them were correctly classified and the remaining 1,443 (17.5%) eployees were miscalssified into class 0 (not-well-paid).

And so on. The overall performances shown that out of the 15,875 employees presented for classification, 13,274 (83.6%) of them were correctly classified leaving only about 16.4% misclassification.

In [ ]:
svc_results = {
    'SVC': [svc_acc, svc_prec, svc_rec]
}
svc_results = pd.DataFrame(svc_results, index=CLASSIFICATION_METRICS)
svc_results

In [ ]:
plt.figure(figsize=(6,4))
svc_results.plot.bar(title='SVC Results')

---
## GradientBoosting Classifier (GBC)

In [ ]:
# instantiation
gb = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.1, criterion='friedman_mse', 
    tol=0.0001, random_state=60)

# training
gb.fit(X_train, y_train)

In [ ]:
# predictions
gb_pred = gb.predict(X_test)

# evaluations
gb_acc = accuracy_score(y_test, gb_pred)
gb_prec = precision_score(y_test, gb_pred, average='weighted')
gb_rec = recall_score(y_test, gb_pred, average='weighted')

gb_cf = confusion_matrix(y_test, gb_pred)

print(f'Classification Report for GradientBoosting-Classifier: \n')
print(classification_report(y_test, gb_pred))

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(gb_cf, fmt='.4g', annot=True, cmap='Blues')
plt.title('\nConfusion Matrix for Gradient Boosting Classifier\n')

In [ ]:
cf_perc = (gb_cf / gb_cf.sum(axis=0) * 100).round(1).reshape(2,2)

plt.figure(figsize=(6,4))
sns.heatmap(cf_perc, annot=True, fmt='.1f', cmap='Blues')
plt.title('\nConfusion Matrix for Gradient Boosting Classifier (%)\n')

**Interpretation of the Confusion Matrix for Gradient Boosting Classifier (GBC)**

As shown above, the x-axis (vertical) indicate the true classes (clusters) and the y-axis (horizontal) indicates the predicted classes (clusters). The diagonal axis indicates number and corresponding percentages of correct classifications, while the rest are misclassifications.

Out of 7,911 employees belonging to class 0 (not-well-paid) presented to the GBC trained model for classification, 7,530 of them were correctly classified, constituting about 88.8% of the population, while 946 (11.2%) employees were mis-classified as belonging to class 1 (well-paid).

Also, of the 7,964 employees who are in class 1 (well-paid), 7,018 (94.9%)of them were correctly classified and the remaining 381 (5.1%) eployees were misclassified as class 0 (not-well-paid).

And so on. The overall performances shown that out of the 15,875 employees presented for classification, 14,548 (92.6%) of them were correctly classified leaving only about 7.4% misclassification.

In [ ]:
gb_results = {
    'GBC': [gb_acc, gb_prec, gb_rec]
}
gb_results = pd.DataFrame(gb_results, index=CLASSIFICATION_METRICS)
gb_results

In [ ]:
gb_results.plot.bar(title='Gradient Boosting Classifier (GBC) Results\n')

# Merged Classification Results


In [ ]:
classification_results = pd.concat([rf_results, svc_results, gb_results], axis='columns')
classification_results

In [ ]:
# Sorting results in order of performance
classification_results.T.sort_values(by='Recall', ascending=False).T

In [ ]:
classification_results.T.sort_values(
    by='Recall', ascending=False).T.plot.bar(
        title='Classification Performance Evaluations')

**Classification Results Interpretation**

As shown in the table, and the associated graph,

We have 91.64%, 91.28% and 83.62% accuracy for Gradient Boosting Classifier (GBC), Random Forest Classifier (RF), and Support Vector Classifier (SVC) respectively.

We have 91.85%, 91.47% and 83.66% precision for Gradient Boosting Classifier (GBC), Random Forest Classifier (RF), and Support Vector Classifier (SVC) respectively.

We have 91.64%, 91.28% and 83.62% recall for Gradient Boosting Classifier (GBC), Random Forest Classifier (RF), and Support Vector Classifier (SVC) respectively.

**It can be observed that overall, Gradient Boosting Classifier (GBC) model out-performed consistently better than all other models across all metrics with about 92% accuracy. Hence, the best performing model, while SVC is the least performing model having about 84% accuracy.**

**It can also be seen that there is only a marginal difference in performance of GBC and RF, as opposed to SVC performances.**

**It should also be noted that the improvement in performances of GBC across all models is significant (> 0.05) compared to other models, even though RF Classifier has a very close performance.**




### Saving trained classification models

In [ ]:
# saving the KMeans trained model
save_model_pickle(rf, filename='RFC_MODEL.pickle')
save_model_pickle(svc, filename='SVC_MODEL.pickle')
save_model_pickle(gb, filename='GBC_MODEL.pickle')

# saving kmeans scaler object
save_model_pickle(scaler_clust, filename='SCALER_for_CLUSTERING.pickle')

# Saving classification results
classification_results.to_csv(f'{RESULTS_PATH}/classification_results.csv')

# Saving a copy of the balanced dataset used for training the classification models
bal_class_ds.to_csv(f'{RESULTS_PATH}/balanced_classification_dataset_for_training.csv')